# Data Cleaning

data is saved on google drive(public)\
https://drive.google.com/drive/folders/1hbWqTjmpvFA-eba4LWN2NbFxJyQQKn3L?usp=sharing

In [1]:
import os
import pandas as pd
import shutil
import tqdm
import numpy as np
import re
import cv2
import torch
import torch.nn as nn
import pytorchvideo
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [187]:
import tensorflow as tf
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')
import gensim, logging
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class

In [195]:
from pytorchvideo.data.encoded_video import EncodedVideo
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import pytorchvideo.transforms
from torch import optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample
)

from torchvision.transforms import (
    Compose,
    Lambda,
    CenterCrop,
    RandomCrop,
    Resize,
    RandomHorizontalFlip
)

In [5]:
path = r'data/'
names = []
labels = []
for dir in os.listdir(path):
    label = dir.replace('.mp4','')
    labels.append(label)
    name = dir
    names.append(name)
dict = {'name':names,'messy label': labels}

In [6]:
df = pd.DataFrame.from_dict(dict)
df

,name,messy label
0,#BACK-[fingerspelled-version].mp4,#BACK-[fingerspelled-version]
1,#what-DO__DO-DO-[lexicalized-fingerspelling].mp4,#what-DO__DO-DO-[lexicalized-fingerspelling]
2,10 dollars _ 10 dollars.mp4,10 dollars _ 10 dollars
3,25 cents _ 25-cents.mp4,25 cents _ 25-cents
4,3 o clock version.mp4,3 o clock version
...,...,...
1691,ZIG-ZAG–(stripes on an object).mp4,ZIG-ZAG–(stripes on an object)
1692,ZIP-LIPS-[zip-your-lips].mp4,ZIP-LIPS-[zip-your-lips]
1693,ZIP-up-[general-version].mp4,ZIP-up-[general-version]
1694,ZIP-up-[jacket version].mp4,ZIP-up-[jacket version]


## Clean the data label

In [7]:
c_data=[]
counts =[]
c_ = 0
data = df['messy label']
for row in data:
    item =''
    c_ +=1
    for i in row:
        if i =='_' or i=='[' or i =='(' or i ==',':
            break
        else:
            item += i

    if re.findall('\w{2}-\w{2}', item):
        item = item.replace('-',' ')
    redic ={'\-\w\-':'-','\-':'-','\#':'#','\@':'@','\–':'–'}
    for pattern in redic:
        if re.findall(pattern, item):
            item = item.replace(redic[pattern],'')

    c_data.append(item.lower())
    counts.append(c_)

In [8]:
df['label']=c_data
df=df.drop('messy label', axis=1)
df.to_csv('ASL Video Data.csv',index=False)
df

,name,label
0,#BACK-[fingerspelled-version].mp4,back
1,#what-DO__DO-DO-[lexicalized-fingerspelling].mp4,what do
2,10 dollars _ 10 dollars.mp4,10 dollars
3,25 cents _ 25-cents.mp4,25 cents
4,3 o clock version.mp4,3 o clock version
...,...,...
1691,ZIG-ZAG–(stripes on an object).mp4,zig zag
1692,ZIP-LIPS-[zip-your-lips].mp4,zip lips
1693,ZIP-up-[general-version].mp4,zip up
1694,ZIP-up-[jacket version].mp4,zip up


In [346]:
for i in range(1691,1696):
    print(i,'&',str(df.iloc[i][0]),'&',str(df.iloc[i][1]),'\\')

1691 & ZIG-ZAG–(stripes on an object).mp4 & zig zag \
1692 & ZIP-LIPS-[zip-your-lips].mp4 & zip lips  \
1693 & ZIP-up-[general-version].mp4 & zip up  \
1694 & ZIP-up-[jacket version].mp4 & zip up  \
1695 & ZIP-[Z-I-P].mp4 & zip \


In [9]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
print('Test:',len(test_set))

Test: 340


In [10]:
train_set, valid_set = train_test_split(train_set, test_size=0.2, random_state=42, shuffle=True)
print('Train:',len(train_set))
print('Valid:',len(valid_set))

Train: 1084
Valid: 272


In [11]:
def getpath(dataset):
    names = list(dataset['name'])
    paths =[]
    for i in range(len(names)):
        name = names[i]
        path = os.path.join('data/', name)
        paths.append(path)
    return paths

In [12]:
trainpath= getpath(train_set)
validpath = getpath(valid_set)
testpath = getpath(test_set)

In [13]:
for i in trainpath:
    source = i
    destination = 'dataset/train/'
    try:
        shutil.copy(source, destination)
        print("File copied successfully.")
 
    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied 

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied 

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied 

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied 

In [14]:
for i in validpath:
    source = i
    destination = 'dataset/valid/'
    try:
        shutil.copy(source, destination)
        print("File copied successfully.")
 
    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied 

In [15]:
for i in testpath:
    source = i
    destination = 'dataset/test/'
    try:
        shutil.copy(source, destination)
        print("File copied successfully.")
 
    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied 

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.


## cleaned the data label into the colum 'label' and stored in the csv file.


## Natural Language Processing

In [189]:
listwords = []
for i in df['label']:
    listwords.append(i.split())

In [203]:
trainlist = []
for i in train_set['label']:
    trainlist.append(i.split())
trainm = Word2Vec(trainlist, min_count=1)
trainwords =  list(trainm.wv.index_to_key)
testlist =[]
for i in test_set['label']:
    testlist.append(i.split())
testm = Word2Vec(testlist, min_count=1)
testwords =  list(testm.wv.index_to_key)

In [247]:
wordsmodel = Word2Vec(listwords, min_count=1)
print(wordsmodel)
words =  list(wordsmodel.wv.index_to_key)
print("first 100 words : ",words[:100])
# access vector for one word
print(wordsmodel.wv['good', 'nice'])
# save model
wordsmodel.save('model.bin')

Word2Vec(vocab=1044, vector_size=100, alpha=0.025)
first 100 words :  ['you', 'your', 'like', 'what', 'how', 'have', 'think', 'sign', 'go', 'many', 'to', 'name', 'class', 'want', 'live', 'up', 'version', 'eat', 'school', 'house', 'deaf', 'do', 'hair', 'before', 'suppose', 'this', 'favorite', 'look', 'city', 'here', 'people', 'me', 'in', 'finish', 'use', 'car', 'past', 'family', 'where', 'hearing', 'teacher', 'dad', 'take', 'learn', 'equal', 'much', 'prefer', 'self', 'year', 'cl', 'all', 'every', 'asl', 'with', 'book', 'night', 'can', 'shirt', 'feel', 'a', 'pizza', 'color', 'dog', 'doctor', 'why', 'movie', 'next', 'for', 'i', 'play', 'sister', 'children', 'law', 'work', 'need', 'earn', 'most', 'student', 'one', 'come', 'phone', 'money', 'backpack', 'tomorrow', 'or', 'yesterday', 'and', 'move', 'weekend', 'mom', 'sometimes', 'from', 'clothes', 'he', 'test', 'bedroom', 'college', 'wish', 'any', 'mind']
[[ 6.1183586e-03 -1.9229189e-03  6.3761356e-03  1.5741596e-03
   4.4058110e-03 -1.05296

# Data Loading

In [191]:
# test with the first data path[0]
cap = cv2.VideoCapture(trainpath[0])
video_fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
print(f"""Frame Per second: {video_fps }
      Total Frames: {total_frames}
      Height: {height} 
      Width: {width}""")


Frame Per second: 29.97002997002997
      Total Frames: 168.0
      Height: 1080.0 
      Width: 1920.0


In [17]:
transform = Compose(
            [
            ApplyTransformToKey(
              key="video",
              transform=Compose(
                  [
                    UniformTemporalSubsample(32),
                    Lambda(lambda x: x / 255.0),
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
#                     RandomShortSideScale(min_size=256, max_size=320),
                    ShortSideScale(size=397),
                    Resize(256),CenterCrop(256),
                    RandomHorizontalFlip(p=0.5),
                  ]
                ),
              ),
            ]
        )

In [319]:
class SegmentationDataset(Dataset):
    def __init__(self, img_dir, label,  transform=None):
        self.img_dir  = img_dir
        self.label = label
        self.transform  = transform
        self.videos = os.listdir(img_dir)
    def __len__(self):
        return len(self.videos)
    def __getitem__(self, index):
        img_path    = os.path.join(self.img_dir, self.videos[index])
        cap = cv2.VideoCapture(img_path)
        video_fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        sampling_rate = 1
        frames_per_second = video_fps
        clip_duration = (total_frames * sampling_rate)/frames_per_second
        start_sec = 0
        end_sec = start_sec + clip_duration
        
        # convert the label str into np array with gensim library
        # filling the caption into the video averagely into 32 frames
        capt =self.label[index]
        c = capt.split()
        caption = []
        for i in c:
            try:
                x = wordsmodel.wv[i]
                x.reshape(1,100)
                caption.append(x)
            except:
                x = wordsmodel.wv['None']
                x.reshape(1,100)
                caption.append(x)
        caption  = np.array(caption)
        num = len(caption)
        mid = num//2
        captions = np.repeat(caption, [32/num], axis =0)
        while len(captions)<32:
            captions = np.append(captions, caption[mid].reshape(1,100), axis =0)
        
        video = EncodedVideo.from_path(img_path)
#         video_data = video
        # Load the desired clip
        video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
        if self.transform is not None:
            augmentations = self.transform(video_data)
            video   = augmentations["video"]
#             audio  = augmentations["audio"]
        if video.shape==torch.Size([3, 32, 256, 256]) and captions.shape ==(32,100):
            return video,  captions

In [320]:
BATCH_SIZE    = 32
captionlist =list(train_set['label'])
traindataset = SegmentationDataset(img_dir='dataset/train', label=captionlist,  transform=transform)
train_loader = DataLoader( traindataset, batch_size=BATCH_SIZE, shuffle=True )

In [321]:
captionlist =list(valid_set['label'])
validdataset = SegmentationDataset(img_dir='dataset/valid', label=captionlist,  transform=transform)
valid_loader = DataLoader( validdataset, batch_size=BATCH_SIZE, shuffle=True )

In [322]:
captionlist =list(test_set['label'])
testdataset = SegmentationDataset(img_dir='dataset/test', label=captionlist,  transform=transform)
test_loader = DataLoader( testdataset, batch_size=BATCH_SIZE, shuffle=True )

In [323]:
traindataset[1][0].shape

torch.Size([3, 32, 256, 256])

In [324]:
traindataset[1][1].shape

(32, 100)

input datasize:\
    32 frames\
    image size [3,256,256]

## first CNN Encoders

In [334]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.conv3 = nn.Conv2d(20, 30, 5)
        self.pool =  nn.MaxPool2d(2)

    def forward(self, i):
#         x = i.view(-1, i.shape[2], i.shape[3], i.shape[4])
        x = i.view(-1, i.shape[0], i.shape[3], i.shape[4])
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = nn.AvgPool2d(4)(x)
        x = x.view(i.shape[0], i.shape[1], -1)
        return x

In [336]:
net_cnn = CNN()
# x = torch.rand(32,1,3,256,256)
x = torch.rand(3, 10, 32, 256, 256)
features = net_cnn(x)
print(features.shape)

torch.Size([3, 10, 15680])


## LSTM Encoder 

In [327]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(1470, 1000)
        self.fc = nn.Linear(1000, 750)
        
    def forward(self, i):
        x, _ = self.lstm(i)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x    
    

In [328]:
net_lstm = LSTM()
out = net_lstm(features)
print(out.shape)

torch.Size([32, 750])


In [329]:
class videocap(nn.Module):
    def __init__(self):
        super(videocap, self).__init__()
        self.layer1 = CNN()
        self.layer2 = LSTM()
    def forward(self, i):
        conv1 = self.layer1(i)
        pool1 = F.max_pool1d(conv1,1)
        out = self.layer2(pool1)
        
        return out

In [330]:
model = videocap()
x = torch.rand(32,1,3,256,256)
model(x).shape

torch.Size([32, 750])

In [ ]:
import tqdm

In [241]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(self.embedding_size, hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state

input_size_encoder = len(trainwords)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = float(0.5)

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(829, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [242]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(self.embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(self.hidden_size, self.output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(testwords)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = float(0.5)
output_size = len(testwords)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)


DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(396, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=396, bias=True)
)


In [243]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(english.vocab)
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state_encoder, cell_state_encoder = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state_decoder, cell_state_decoder = self.Decoder_LSTM(x, hidden_state_encoder, cell_state_encoder)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs
Model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
print(Model)

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(829, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(396, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=396, bias=True)
  )
)


In [337]:
for epoch in range(epochs):
    print(f"Epoch{epoch +1} of {epochs}")
    model.train()

    for bi, data in tqdm.tqdm(enumerate(train_loader), total = len(train_loader)):
        vid = data[0] # input image
        labels = data[1].to(device).float() # input boxes
        out = model(vid.to(device))  # output
        label, box = out 
        # define the loss function
        loss  = torch.sqrt(loss_1(box, boxes))*0.01 + loss_2(label.squeeze(0), labels) 
        # save all loss numbers by steps

Epoch1 of 2


  0%|                                                                                           | 0/34 [00:30<?, ?it/s]


RuntimeError: Given groups=1, weight of size [10, 3, 5, 5], expected input[96, 32, 256, 256] to have 3 channels, but got 32 channels instead

### NLP

In [ ]:
x= list(train_set['label'])[0].split()
x

In [ ]:
label =[]
for i in x:
    label.append(wv[i])

In [ ]:
x = np.array(label)

In [ ]:
ten = tf.convert_to_tensor(x)

In [ ]:
model = Word2Vec.load("word2vec.model")
model.train([["pilot use sun glasses"]], total_examples=1, epochs=1)

In [ ]:
vector = model.wv['computer']
vector.shape

In [ ]:
def get_w2vdf(df):
    w2v_df = pd.DataFrame(train_set['label']).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = w2v_df[i][0].split(" ")
    return w2v_df

def train_w2v(w2v_df):
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=4,
                         window=4,
                         size=300, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         sg = 1,
                         workers=cores-1)
    
    w2v_model.build_vocab(w2v_df, progress_per=10000)
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model

In [ ]:
w2v_model["great"]

In [ ]:
captionlist =list(train_set['label'])
captionlist[0]

In [ ]:
vector = model.wv['computer']
vector

In [ ]:
w2v_df = get_w2vdf(df)
w2v_model = train_w2v(w2v_df)

In [ ]:
# Load vocabulary
with open('', 'rb') as f:
    vocab = pickle.load(f)

In [ ]:
torchtext.vocab.Vocab(vocab)

## Attention LSTM Decoder

In [ ]:
class AttentionModel(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
        super(AttentionModel, self).__init__()


        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length

        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
        self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
        self.lstm = nn.LSTM(embedding_length, hidden_size)
        self.label = nn.Linear(hidden_size, output_size)
        #self.attn_fc_layer = nn.Linear()

    def attention_net(self, lstm_output, final_state):


        hidden = final_state.squeeze(0)
        attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
        soft_attn_weights = F.softmax(attn_weights, 1)
        new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)

        return new_hidden_state

    def forward(self, input_sentences, batch_size=None):

        input = self.word_embeddings(input_sentences)
        input = input.permute(1, 0, 2)
        if batch_size is None:
            h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())

        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0)) # final_hidden_state.size() = (1, batch_size, hidden_size) 
        output = output.permute(1, 0, 2) # output.size() = (batch_size, num_seq, hidden_size)

        attn_output = self.attention_net(output, final_hidden_state)
        logits = self.label(attn_output)

        return logits


In [ ]:
# Attention Decoder
####################
class Decoder(nn.Module):

    def __init__(self, vocab_size, use_glove, use_bert):
        super(Decoder, self).__init__()
        self.encoder_dim = 2048
        self.attention_dim = 512
        self.use_bert = use_bert

        if use_glove:
            self.embed_dim = 300
        elif use_bert:
            self.embed_dim = 768
        else:
            self.embed_dim = 512

        self.decoder_dim = 512
        self.vocab_size = vocab_size
        self.dropout = 0.5
        
        # soft attention
        self.enc_att = nn.Linear(2048, 512)
        self.dec_att = nn.Linear(512, 512)
        self.att = nn.Linear(512, 1)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

        # decoder layers
        self.dropout = nn.Dropout(p=self.dropout)
        self.decode_step = nn.LSTMCell(self.embed_dim + self.encoder_dim, self.decoder_dim, bias=True)
        self.h_lin = nn.Linear(self.encoder_dim, self.decoder_dim)
        self.c_lin = nn.Linear(self.encoder_dim, self.decoder_dim)
        self.f_beta = nn.Linear(self.decoder_dim, self.encoder_dim)
        self.sigmoid = nn.Sigmoid()
        self.fc = nn.Linear(self.decoder_dim, self.vocab_size)

        # init variables
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)
        
        if not use_bert:
            self.embedding = nn.Embedding(vocab_size, self.embed_dim)
            self.embedding.weight.data.uniform_(-0.1, 0.1)

            # load Glove embeddings
            if use_glove:
                self.embedding.weight = nn.Parameter(glove_vectors)

            # always fine-tune embeddings (even with GloVe)
            for p in self.embedding.parameters():
                p.requires_grad = True

    def forward(self, encoder_out, encoded_captions, caption_lengths):    
        batch_size = encoder_out.size(0)
        encoder_dim = encoder_out.size(-1)
        vocab_size = self.vocab_size
        dec_len = [x-1 for x in caption_lengths]
        max_dec_len = max(dec_len)

        encoder_out = encoder_out.view(batch_size, -1, encoder_dim)
        num_pixels = encoder_out.size(1)

        # load bert or regular embeddings
        if not self.use_bert:
            embeddings = self.embedding(encoded_captions)
        elif self.use_bert:
            embeddings = []
            for cap_idx in  encoded_captions:
                
                # padd caption to correct size
                while len(cap_idx) < max_dec_len:
                    cap_idx.append(PAD)
                    
                cap = ' '.join([vocab.idx2word[word_idx.item()] for word_idx in cap_idx])
                cap = u'[CLS] '+cap
                
                tokenized_cap = tokenizer.tokenize(cap)                
                indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_cap)
                tokens_tensor = torch.tensor([indexed_tokens]).to(device)

                with torch.no_grad():
                    encoded_layers, _ = BertModel(tokens_tensor)

                bert_embedding = encoded_layers[11].squeeze(0)
                
                split_cap = cap.split()
                tokens_embedding = []
                j = 0

                for full_token in split_cap:
                    curr_token = ''
                    x = 0
                    for i,_ in enumerate(tokenized_cap[1:]): # disregard CLS
                        token = tokenized_cap[i+j]
                        piece_embedding = bert_embedding[i+j]
                        
                        # full token
                        if token == full_token and curr_token == '' :
                            tokens_embedding.append(piece_embedding)
                            j += 1
                            break
                        else: # partial token
                            x += 1
                            
                            if curr_token == '':
                                tokens_embedding.append(piece_embedding)
                                curr_token += token.replace('#', '')
                            else:
                                tokens_embedding[-1] = torch.add(tokens_embedding[-1], piece_embedding)
                                curr_token += token.replace('#', '')
                                
                                if curr_token == full_token: # end of partial
                                    j += x
                                    break                            

                cap_embedding = torch.stack(tokens_embedding)
                embeddings.append(cap_embedding)
  
            embeddings = torch.stack(embeddings)

        # init hidden state
        avg_enc_out = encoder_out.mean(dim=1)
        h = self.h_lin(avg_enc_out)
        c = self.c_lin(avg_enc_out)

        predictions = torch.zeros(batch_size, max_dec_len, vocab_size).to(device)
        alphas = torch.zeros(batch_size, max_dec_len, num_pixels).to(device)

        for t in range(max(dec_len)):
            batch_size_t = sum([l > t for l in dec_len ])
            
            # soft-attention
            enc_att = self.enc_att(encoder_out[:batch_size_t])
            dec_att = self.dec_att(h[:batch_size_t])
            att = self.att(self.relu(enc_att + dec_att.unsqueeze(1))).squeeze(2)
            alpha = self.softmax(att)
            attention_weighted_encoding = (encoder_out[:batch_size_t] * alpha.unsqueeze(2)).sum(dim=1)
        
            gate = self.sigmoid(self.f_beta(h[:batch_size_t]))
            attention_weighted_encoding = gate * attention_weighted_encoding
            
            batch_embeds = embeddings[:batch_size_t, t, :]            
            cat_val = torch.cat([batch_embeds.double(), attention_weighted_encoding.double()], dim=1)
            
            h, c = self.decode_step(cat_val.float(),(h[:batch_size_t].float(), c[:batch_size_t].float()))
            preds = self.fc(self.dropout(h))
            predictions[:batch_size_t, t, :] = preds
            alphas[:batch_size_t, t, :] = alpha
            
        # preds, sorted capts, dec lens, attention wieghts
        return predictions, encoded_captions, dec_len, alphas


In [ ]:
# k = []
# for i in range(len(traindataset)):
#     s = traindataset[i][0].shape
#     if s not in k:
#         k.append(s)
#         print(s)
#     else:
#         pass

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
        
#         self.conv1 = nn.Conv2d(3, 10, 5)
#         self.conv2 = nn.Conv2d(10, 20, 5)
#         self.conv3 = nn.Conv2d(20, 30, 5)
        
#     def forward(self, i):
#         x = i.view(-1, i.shape[2], i.shape[3], i.shape[4])
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
#         x = nn.AvgPool2d(4)(x)
#         x = x.view(i.shape[0], i.shape[1], -1)
#         return x
    
# class LSTM(nn.Module):
#     def __init__(self):
#         super(LSTM, self).__init__()
#         self.lstm = nn.LSTM(750, 100)
#         self.fc = nn.Linear(100*50, 2)
        
#     def forward(self, x):
#         x, _ = self.lstm(x)
#         x = x.view(x.shape[0], -1)
#         x = self.fc(x)
#         return x    
    
# net_cnn = CNN()
# net_lstm = LSTM()


In [ ]:
# features = net_cnn(x)
# out = net_lstm(features)

In [ ]:
device  = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True).to(device)
# model.blocks[5].proj = nn.Linear(2048, 2).to(device)
model = model.to(device)

In [ ]:
model

In [ ]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
#             NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCrop(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

In [ ]:
video_path = testpath[0]

In [ ]:
start_sec = 0
end_sec = start_sec + clip_duration

# Initialize an EncodedVideo helper class
video = EncodedVideo.from_path(video_path)

# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# Apply a transform to normalize the video input
video_data = transform(video_data)

# Move the inputs to the desired device
inputs = video_data["video"]
inputs[0].shape

In [ ]:
inputs = [i.to(device)[None, ...] for i in inputs]
inputs[0].shape

In [ ]:
preds = model(inputs)

In [ ]:
# Get the predicted classes
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=5).indices

In [ ]:
print(pred_classes)

In [ ]:
preds.shape

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
        super(LSTMClassifier, self).__init__()



        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length

        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)# Initializing the look-up table.
        self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False) # Assigning the look-up table to the pre-trained GloVe word embedding.
        self.lstm = nn.LSTM(embedding_length, hidden_size)
        self.label = nn.Linear(hidden_size, output_size)

    def forward(self, input_sentence, batch_size=None):
        input = self.word_embeddings(input_sentence) # embedded input of shape = (batch_size, num_sequences,  embedding_length)
        input = input.permute(1, 0, 2) # input.size() = (num_sequences, batch_size, embedding_length)
        if batch_size is None:
            h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) # Initial hidden state of the LSTM
            c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) # Initial cell state of the LSTM
        else:
            h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        final_output = self.label(final_hidden_state[-1]) # final_hidden_state.size() = (1, batch_size, hidden_size) & final_output.size() = (batch_size, output_size)

        return final_output

In [ ]:
class AttentionModel(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
        super(AttentionModel, self).__init__()


        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length

        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
        self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
        self.lstm = nn.LSTM(embedding_length, hidden_size)
        self.label = nn.Linear(hidden_size, output_size)
        #self.attn_fc_layer = nn.Linear()

    def attention_net(self, lstm_output, final_state):


        hidden = final_state.squeeze(0)
        attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
        soft_attn_weights = F.softmax(attn_weights, 1)
        new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)

        return new_hidden_state

    def forward(self, input_sentences, batch_size=None):

        input = self.word_embeddings(input_sentences)
        input = input.permute(1, 0, 2)
        if batch_size is None:
            h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())

        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0)) # final_hidden_state.size() = (1, batch_size, hidden_size) 
        output = output.permute(1, 0, 2) # output.size() = (batch_size, num_seq, hidden_size)

        attn_output = self.attention_net(output, final_hidden_state)
        logits = self.label(attn_output)

        return logits


In [ ]:
print(f'Number of training examples: {len(train_set)}')
print(f'Number of testing examples: {len(valid_set)}')

In [ ]:
from torchtext import data

In [ ]:
import torchtext
import spacy
# from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator
import torchtext.data as data

In [ ]:
TEXT = data.Field()
LABELS = data.Field()

In [ ]:
TEXT.build_vocab(train_set)
LABELS.build_vocab(train_set)

In [ ]:
tx

In [ ]:
# Map the predicted classes to the label names
pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes[0]]
print("Predicted labels: %s" % ", ".join(pred_class_names))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(2):
    model.train()
    for bi, data in tqdm.tqdm(enumerate(train_loader), total = len(train_loader)):
        
        inputs =  data[0].to(device)
#         labels =  data[1].to(device)

#         optimizer.zero_grad()

# #         with torch.set_grad_enabled:
        outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         print(type(preds))
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()



# Video Captioning

We try video caption models and see the result in different words performance\
noun, adjective, adverb, article, preposition, verb, pronoun, conjunction.\
model\
LSTM-YT, S2VT, H-RNN, DenseCap-event\

To evaluate the generated results, we first employ four different
traditional metrics: Bleu (B) [26], METEOR (M) [20], CIDEr-D
(C) [35], SPICE (S) [1] and Rouge-L (R) [2